In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.375
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from geocat.comp import eofunc_eofs, eofunc_pcs
from datetime import datetime
import warnings
from matplotlib.patches import Rectangle
import random
import numba
import statsmodels.api as sm
from sklearn.metrics import r2_score
import pandas as pd
import os
import sys
import glob

# Step 1: Add parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "/scratch/ivyglade/pdo/"))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Step 2: Now you can import file.py
import pdo_functions  # assuming file.py contains functions/classes

import importlib
importlib.reload(pdo_functions)

In [ ]:
# Open data
u_200 = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_large_scale_envt/u_200_monthly/u_200_mam_1940-2024.nc')['__xarray_dataarray_variable__']
v_200 = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_large_scale_envt/v_200_monthly/v_200_mam_1940-2024.nc')['__xarray_dataarray_variable__']
u_850 = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_large_scale_envt/u_850_monthly/u_850_mam_1940-2024.nc')['__xarray_dataarray_variable__']
v_850 = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_large_scale_envt/v_850_monthly/v_850_mam_1940-2024.nc')['__xarray_dataarray_variable__']
z_500 = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_large_scale_envt/Z_500_monthly/Z_500_mam_1940-2024.nc')['__xarray_dataarray_variable__']
tcwv  = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_large_scale_envt/tcwv_monthly/tcwv_monthly_mam_1940-2024.nc')['TCWV']

In [ ]:
tcwv_conus = tcwv.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))

In [ ]:
tcwv_conus_mam = tcwv_conus.sel(time=tcwv_conus.time.dt.month.isin([3, 4, 5]))

In [ ]:
# Resample to seasonal averages
u_200_szn = u_200.resample(time='YE').mean()
v_200_szn = v_200.resample(time='YE').mean()
u_850_szn = u_850.resample(time='YE').mean()
v_850_szn = v_850.resample(time='YE').mean()
z_500_szn = z_500.resample(time='YE').mean()
tcwv_szn = tcwv_conus_mam.resample(time='YE').mean()

In [ ]:
# Calculate mean wind
wind_200 = np.sqrt(u_200_szn **2 + v_200_szn **2)
wind_850 = np.sqrt(u_850_szn **2 + v_850_szn **2)

In [ ]:
# Detrend
wind_200_de = pdo_functions.detrend_dim(wind_200, 'time', 2)
wind_850_de = pdo_functions.detrend_dim(wind_850, 'time', 2)
z_500_de = pdo_functions.detrend_dim(z_500_szn, 'time', 2)
u_850_de = pdo_functions.detrend_dim(u_850_szn, 'time', 2)
v_850_de = pdo_functions.detrend_dim(v_850_szn, 'time', 2)
tcwv_de = pdo_functions.detrend_dim(tcwv_szn, 'time', 2)

In [ ]:
# Open SST data and compute the PDO and Nino3.4
sst = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/HadISST_sst.nc')['sst']

pdo = pdo_functions.pdo_from_hadisst(sst, 1870, 2024)

oni = pdo_functions.oni_moving_base(sst)

In [ ]:
# 3- month rolling mean
oni_rolling = oni.rolling(time=3, center=True).mean()
oni_rolling_1940_2024 = oni_rolling.sel(time=oni_rolling.time.dt.year.isin(np.arange(1940, 2025, 1)))
oni_rolling_1940_2024_mam = oni_rolling_1940_2024.sel(time=oni_rolling_1940_2024.time.dt.month.isin([3, 4, 5])).resample(time='YE').mean()

# Calculating when El Nino and La Nina events occur
oni_calc = np.zeros((1668))
for i in range(1668):
    if i < 5:
        continue
    else:
        if (oni_rolling[(i-4)] > 0.5) & (oni_rolling[(i-3)] > 0.5) & (oni_rolling[(i-2)] > 0.5) & (oni_rolling[(i-1)] > 0.5) & (oni_rolling[i] > 0.5):
            oni_calc[i] = 1
        elif (oni_rolling[(i-4)] < -0.5) & (oni_rolling[(i-3)] < -0.5) & (oni_rolling[(i-2)] < -0.5) & (oni_rolling[(i-1)] < -0.5) & (oni_rolling[i] < -0.5):
            oni_calc[i] = -1
        else:
            continue

# Coonvert to XR
oni_calc_xr = xr.DataArray(oni_calc, coords={'time':oni['time']}, dims=['time'])

# 1940-2024 only
oni_1940_2024 = oni_calc_xr.sel(time=oni_calc_xr.time.dt.year.isin(np.arange(1940, 2025, 1)))
oni_1940_2024_mam = oni_1940_2024.sel(time=oni_1940_2024.time.dt.month.isin([3, 4, 5]))

# Need to calculate when El Nino/La Nina Occurs in a given year
oni_1940_2024_mam_ann = np.zeros((85))
for i in range(85):
    if oni_1940_2024_mam[i*3: (i+1)*3].sum(axis=0) > 0:
        oni_1940_2024_mam_ann[i] = 1

    elif oni_1940_2024_mam[i*3: (i+1)*3].sum(axis=0) < 0:
        oni_1940_2024_mam_ann[i] = -1

    else:
        continue

In [ ]:
# Convert pdo to xarray
pdo_xr = xr.DataArray(pdo, coords={'time':sst['time']}, dims=['time'])

# Subset only 1940-2024
pdo_1940_2024 = pdo_xr.sel(time=pdo_xr.time.dt.year.isin(np.arange(1940, 2025, 1)))

# only MAM and take seasonal averages
pdo_1940_2024_mam = pdo_1940_2024.sel(time=pdo_1940_2024.time.dt.month.isin([3, 4, 5])).resample(time='YE').mean()

# aligning time arrays with that of CAPE
pdo_1940_2024_mam['time'] = z_500_szn['time']

oni_rolling_1940_2024_mam['time'] = z_500_szn['time']

oni_ann_xr = xr.DataArray(oni_1940_2024_mam_ann, coords={'time':z_500_szn['time']}, dims=['time'])

In [ ]:
# Compositing
wind_200_pos_pdo = xr.where(pdo_1940_2024_mam > 0, wind_200_de, np.nan).dropna(dim='time')
wind_200_neg_pdo = xr.where(pdo_1940_2024_mam < 0, wind_200_de, np.nan).dropna(dim='time')

wind_200_pos_oni = xr.where(oni_rolling_1940_2024_mam > 0, wind_200_de, np.nan).dropna(dim='time')
wind_200_neg_oni = xr.where(oni_rolling_1940_2024_mam < 0, wind_200_de, np.nan).dropna(dim='time')

wind_200_pos_nino = xr.where(oni_ann_xr == 1, wind_200_de, np.nan).dropna(dim='time')
wind_200_neg_nina = xr.where(oni_ann_xr == -1, wind_200_de, np.nan).dropna(dim='time')

wind_200_pos_pdo_neut_oni = xr.where((pdo_1940_2024_mam > 0) & (oni_ann_xr == 0), wind_200_de, np.nan).dropna(dim='time')
wind_200_neg_pdo_neut_oni = xr.where((pdo_1940_2024_mam < 0) & (oni_ann_xr == 0), wind_200_de, np.nan).dropna(dim='time')

wind_850_pos_pdo = xr.where(pdo_1940_2024_mam > 0, wind_850_de, np.nan).dropna(dim='time')
wind_850_neg_pdo = xr.where(pdo_1940_2024_mam < 0, wind_850_de, np.nan).dropna(dim='time')

wind_850_pos_oni = xr.where(oni_rolling_1940_2024_mam > 0, wind_850_de, np.nan).dropna(dim='time')
wind_850_neg_oni = xr.where(oni_rolling_1940_2024_mam < 0, wind_850_de, np.nan).dropna(dim='time')

wind_850_pos_nino = xr.where(oni_ann_xr == 1, wind_850_de, np.nan).dropna(dim='time')
wind_850_neg_nina = xr.where(oni_ann_xr == -1, wind_850_de, np.nan).dropna(dim='time')

wind_850_pos_pdo_neut_oni = xr.where((pdo_1940_2024_mam > 0) & (oni_ann_xr == 0), wind_850_de, np.nan).dropna(dim='time')
wind_850_neg_pdo_neut_oni = xr.where((pdo_1940_2024_mam < 0) & (oni_ann_xr == 0), wind_850_de, np.nan).dropna(dim='time')

z_500_pos_pdo = xr.where(pdo_1940_2024_mam > 0, z_500_de, np.nan).dropna(dim='time')
z_500_neg_pdo = xr.where(pdo_1940_2024_mam < 0, z_500_de, np.nan).dropna(dim='time')

z_500_pos_oni = xr.where(oni_rolling_1940_2024_mam > 0, z_500_de, np.nan).dropna(dim='time')
z_500_neg_oni = xr.where(oni_rolling_1940_2024_mam < 0, z_500_de, np.nan).dropna(dim='time')

z_500_pos_nino = xr.where(oni_ann_xr == 1, z_500_de, np.nan).dropna(dim='time')
z_500_neg_nina = xr.where(oni_ann_xr == -1, z_500_de, np.nan).dropna(dim='time')

z_500_pos_pdo_neut_oni = xr.where((pdo_1940_2024_mam > 0) & (oni_ann_xr == 0), z_500_de, np.nan).dropna(dim='time')
z_500_neg_pdo_neut_oni = xr.where((pdo_1940_2024_mam < 0) & (oni_ann_xr == 0), z_500_de, np.nan).dropna(dim='time')

u_850_pos_pdo = xr.where(pdo_1940_2024_mam > 0, u_850_de, np.nan).dropna(dim='time')*land
u_850_neg_pdo = xr.where(pdo_1940_2024_mam < 0, u_850_de, np.nan).dropna(dim='time')*land

u_850_pos_oni = xr.where(oni_rolling_1940_2024_mam > 0, u_850_de, np.nan).dropna(dim='time')*land
u_850_neg_oni = xr.where(oni_rolling_1940_2024_mam < 0, u_850_de, np.nan).dropna(dim='time')*land

u_850_pos_nino = xr.where(oni_ann_xr == 1, u_850_de, np.nan).dropna(dim='time')*land
u_850_neg_nina = xr.where(oni_ann_xr == -1, u_850_de, np.nan).dropna(dim='time')*land

u_850_pos_pdo_neut_oni = xr.where((pdo_1940_2024_mam > 0) & (oni_ann_xr == 0), u_850_de, np.nan).dropna(dim='time')*land
u_850_neg_pdo_neut_oni = xr.where((pdo_1940_2024_mam < 0) & (oni_ann_xr == 0), u_850_de, np.nan).dropna(dim='time')*land

v_850_pos_pdo = xr.where(pdo_1940_2024_mam > 0, v_850_de, np.nan).dropna(dim='time')*land
v_850_neg_pdo = xr.where(pdo_1940_2024_mam < 0, v_850_de, np.nan).dropna(dim='time')*land

v_850_pos_oni = xr.where(oni_rolling_1940_2024_mam > 0, v_850_de, np.nan).dropna(dim='time')*land
v_850_neg_oni = xr.where(oni_rolling_1940_2024_mam < 0, v_850_de, np.nan).dropna(dim='time')*land

v_850_pos_nino = xr.where(oni_ann_xr == 1, v_850_de, np.nan).dropna(dim='time')*land
v_850_neg_nina = xr.where(oni_ann_xr == -1, v_850_de, np.nan).dropna(dim='time')*land

v_850_pos_pdo_neut_oni = xr.where((pdo_1940_2024_mam > 0) & (oni_ann_xr == 0), v_850_de, np.nan).dropna(dim='time')*land
v_850_neg_pdo_neut_oni = xr.where((pdo_1940_2024_mam < 0) & (oni_ann_xr == 0), v_850_de, np.nan).dropna(dim='time')*land

tcwv_pos_pdo = xr.where(pdo_1940_2024_mam > 0, tcwv_de, np.nan).dropna(dim='time')
tcwv_neg_pdo = xr.where(pdo_1940_2024_mam < 0, tcwv_de, np.nan).dropna(dim='time')

tcwv_pos_oni = xr.where(oni_rolling_1940_2024_mam > 0, tcwv_de, np.nan).dropna(dim='time')
tcwv_neg_oni = xr.where(oni_rolling_1940_2024_mam < 0, tcwv_de, np.nan).dropna(dim='time')

tcwv_pos_nino = xr.where(oni_ann_xr == 1, tcwv_de, np.nan).dropna(dim='time')
tcwv_neg_nina = xr.where(oni_ann_xr == -1, tcwv_de, np.nan).dropna(dim='time')

tcwv_pos_pdo_neut_oni = xr.where((pdo_1940_2024_mam > 0) & (oni_ann_xr == 0), tcwv_de, np.nan).dropna(dim='time')
tcwv_neg_pdo_neut_oni = xr.where((pdo_1940_2024_mam < 0) & (oni_ann_xr == 0), tcwv_de, np.nan).dropna(dim='time')

In [ ]:
def bootstrap_p_values_fast(sample_mean, cape_climo, comp_cond, n_boot=10_000):
    n_lats, n_lons, n_time = cape_climo.shape
    n_comp = np.count_nonzero(comp_cond)

    # Flatten the spatial dimensions
    grid_size = n_lats * n_lons
    cape_flat = cape_climo.reshape(grid_size, n_time)
    sample_mean_flat = sample_mean.ravel()

    # Draw bootstrap samples for all grid points at once
    rand_idx = np.random.randint(0, n_time, size=(n_boot, n_comp))
    boot_samples = cape_flat[:, rand_idx]  # shape: (grid_size, n_boot, n_comp)

    # Mean across composite months
    boot_means = boot_samples.mean(axis=2)  # shape: (grid_size, n_boot)

    # Climatology mean for each grid point
    climo_mean = cape_flat.mean(axis=1)  # shape: (grid_size,)

    # Calculate p-values
    diffs = np.abs(boot_means - climo_mean[:, None])  # shape: (grid_size, n_boot)
    test_diffs = np.abs(sample_mean_flat - climo_mean)
    p_values_flat = np.mean(diffs >= test_diffs[:, None], axis=1)

    # Confidence intervals
    lower_flat = np.percentile(boot_means, 2.5, axis=1)
    upper_flat = np.percentile(boot_means, 97.5, axis=1)

    # Reshape back to (lat, lon)
    p_values = p_values_flat.reshape(n_lats, n_lons)
    lower_grid = lower_flat.reshape(n_lats, n_lons)
    upper_grid = upper_flat.reshape(n_lats, n_lons)

    return p_values, upper_grid, lower_grid

In [ ]:
wind_200_pos_pdo_p_fast, wind_200_pos_pdo_upper_fast, wind_200_pos_pdo_lower_fast = bootstrap_p_values_fast(wind_200_pos_pdo.values.mean(axis=0), wind_200_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values > 0)

In [ ]:
wind_200_pos_pdo_neut_nino_p_fast, wind_200_pos_pdo_neut_nino_upper_fast, wind_200_pos_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(wind_200_pos_pdo_neut_oni.values.mean(axis=0), wind_200_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam > 0) & (oni_ann_xr == 0))

In [ ]:
wind_200_neg_pdo_p_fast, wind_200_neg_pdo_upper_fast, wind_200_neg_pdo_lower_fast = bootstrap_p_values_fast(wind_200_neg_pdo.values.mean(axis=0), wind_200_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values < 0)

In [ ]:
wind_200_neg_pdo_neut_nino_p_fast, wind_200_neg_pdo_neut_nino_upper_fast, wind_200_neg_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(wind_200_neg_pdo_neut_oni.values.mean(axis=0), wind_200_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam < 0) & (oni_ann_xr == 0))

In [ ]:
wind_200_pos_nino_p_fast, wind_200_pos_nino_upper_fast, wind_200_pos_nino_lower_fast = bootstrap_p_values_fast(wind_200_pos_nino.values.mean(axis=0), wind_200_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == 1)

In [ ]:
wind_200_neg_nino_p_fast, wind_200_neg_nino_upper_fast, wind_200_neg_nino_lower_fast = bootstrap_p_values_fast(wind_200_neg_nina.values.mean(axis=0), wind_200_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == -1)

In [ ]:
wind_200_pos_oni_p_fast, wind_200_pos_oni_upper_fast, wind_200_pos_oni_lower_fast = bootstrap_p_values_fast(wind_200_pos_oni.values.mean(axis=0), wind_200_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam > 0)

In [ ]:
wind_200_neg_oni_p_fast, wind_200_neg_oni_upper_fast, wind_200_neg_oni_lower_fast = bootstrap_p_values_fast(wind_200_neg_oni.values.mean(axis=0), wind_200_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam < 0)

In [ ]:
## Spacer ##

In [ ]:
wind_850_pos_pdo_p_fast, wind_850_pos_pdo_upper_fast, wind_850_pos_pdo_lower_fast = bootstrap_p_values_fast(wind_850_pos_pdo.values.mean(axis=0), wind_850_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values > 0)

In [ ]:
wind_850_pos_pdo_neut_nino_p_fast, wind_850_pos_pdo_neut_nino_upper_fast, wind_850_pos_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(wind_850_pos_pdo_neut_oni.values.mean(axis=0), wind_850_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam > 0) & (oni_ann_xr == 0))

In [ ]:
wind_850_neg_pdo_p_fast, wind_850_neg_pdo_upper_fast, wind_850_neg_pdo_lower_fast = bootstrap_p_values_fast(wind_850_neg_pdo.values.mean(axis=0), wind_850_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values < 0)

In [ ]:
wind_850_neg_pdo_neut_nino_p_fast, wind_850_neg_pdo_neut_nino_upper_fast, wind_850_neg_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(wind_850_neg_pdo_neut_oni.values.mean(axis=0), wind_850_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam < 0) & (oni_ann_xr == 0))

In [ ]:
wind_850_pos_nino_p_fast, wind_850_pos_nino_upper_fast, wind_850_pos_nino_lower_fast = bootstrap_p_values_fast(wind_850_pos_nino.values.mean(axis=0), wind_850_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == 1)

In [ ]:
wind_850_neg_nino_p_fast, wind_850_neg_nino_upper_fast, wind_850_neg_nino_lower_fast = bootstrap_p_values_fast(wind_850_neg_nina.values.mean(axis=0), wind_850_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == -1)

In [ ]:
wind_850_pos_oni_p_fast, wind_850_pos_oni_upper_fast, wind_850_pos_oni_lower_fast = bootstrap_p_values_fast(wind_850_pos_oni.values.mean(axis=0), wind_850_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam > 0)

In [ ]:
wind_850_neg_oni_p_fast, wind_850_neg_oni_upper_fast, wind_850_neg_oni_lower_fast = bootstrap_p_values_fast(wind_850_neg_oni.values.mean(axis=0), wind_850_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam < 0)

In [ ]:
## Spacer ##

In [ ]:
z_500_pos_pdo_p_fast, z_500_pos_pdo_upper_fast, z_500_pos_pdo_lower_fast = bootstrap_p_values_fast(z_500_pos_pdo.values.mean(axis=0), z_500_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values > 0)

In [ ]:
z_500_pos_pdo_neut_nino_p_fast, z_500_pos_pdo_neut_nino_upper_fast, z_500_pos_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(z_500_pos_pdo_neut_oni.values.mean(axis=0), z_500_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam > 0) & (oni_ann_xr == 0))

In [ ]:
z_500_neg_pdo_p_fast, z_500_neg_pdo_upper_fast, z_500_neg_pdo_lower_fast = bootstrap_p_values_fast(z_500_neg_pdo.values.mean(axis=0), z_500_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values < 0)

In [ ]:
z_500_neg_pdo_neut_nino_p_fast, z_500_neg_pdo_neut_nino_upper_fast, z_500_neg_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(z_500_neg_pdo_neut_oni.values.mean(axis=0), z_500_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam < 0) & (oni_ann_xr == 0))

In [ ]:
z_500_pos_nino_p_fast, z_500_pos_nino_upper_fast, z_500_pos_nino_lower_fast = bootstrap_p_values_fast(z_500_pos_nino.values.mean(axis=0), z_500_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == 1)

In [ ]:
z_500_neg_nino_p_fast, z_500_neg_nino_upper_fast, z_500_neg_nino_lower_fast = bootstrap_p_values_fast(z_500_neg_nina.values.mean(axis=0), z_500_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == -1)

In [ ]:
z_500_pos_oni_p_fast, z_500_pos_oni_upper_fast, z_500_pos_oni_lower_fast = bootstrap_p_values_fast(z_500_pos_oni.values.mean(axis=0), z_500_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam > 0)

In [ ]:
z_500_neg_oni_p_fast, z_500_neg_oni_upper_fast, z_500_neg_oni_lower_fast = bootstrap_p_values_fast(z_500_neg_oni.values.mean(axis=0), z_500_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam < 0)

In [ ]:
## Spacer ##

In [ ]:
wind_200_pos_pdo_adj_p = pdo_functions.control_FDR(wind_200_pos_pdo_p_fast, 101, 237, 0.1)
wind_200_neg_pdo_adj_p = pdo_functions.control_FDR(wind_200_neg_pdo_p_fast, 101, 237, 0.1)
wind_200_pos_pdo_neut_nino_adj_p = pdo_functions.control_FDR(wind_200_pos_pdo_neut_nino_p_fast, 101, 237, 0.1)
wind_200_neg_pdo_neut_nino_adj_p = pdo_functions.control_FDR(wind_200_neg_pdo_neut_nino_p_fast, 101, 237, 0.1)
wind_200_pos_oni_adj_p = pdo_functions.control_FDR(wind_200_pos_oni_p_fast, 101, 237, 0.1)
wind_200_neg_oni_adj_p = pdo_functions.control_FDR(wind_200_neg_oni_p_fast, 101, 237, 0.1)
wind_200_pos_nino_adj_p = pdo_functions.control_FDR(wind_200_pos_nino_p_fast, 101, 237, 0.1)
wind_200_neg_nino_adj_p = pdo_functions.control_FDR(wind_200_neg_nino_p_fast, 101, 237, 0.1)

wind_850_pos_pdo_adj_p = pdo_functions.control_FDR(wind_850_pos_pdo_p_fast, 101, 237, 0.1)
wind_850_neg_pdo_adj_p = pdo_functions.control_FDR(wind_850_neg_pdo_p_fast, 101, 237, 0.1)
wind_850_pos_pdo_neut_nino_adj_p = pdo_functions.control_FDR(wind_850_pos_pdo_neut_nino_p_fast, 101, 237, 0.1)
wind_850_neg_pdo_neut_nino_adj_p = pdo_functions.control_FDR(wind_850_neg_pdo_neut_nino_p_fast, 101, 237, 0.1)
wind_850_pos_oni_adj_p = pdo_functions.control_FDR(wind_850_pos_oni_p_fast, 101, 237, 0.1)
wind_850_neg_oni_adj_p = pdo_functions.control_FDR(wind_850_neg_oni_p_fast, 101, 237, 0.1)
wind_850_pos_nino_adj_p = pdo_functions.control_FDR(wind_850_pos_nino_p_fast, 101, 237, 0.1)
wind_850_neg_nino_adj_p = pdo_functions.control_FDR(wind_850_neg_nino_p_fast, 101, 237, 0.1)

z_500_pos_pdo_adj_p = pdo_functions.control_FDR(z_500_pos_pdo_p_fast, 101, 237, 0.1)
z_500_neg_pdo_adj_p = pdo_functions.control_FDR(z_500_neg_pdo_p_fast, 101, 237, 0.1)
z_500_pos_pdo_neut_nino_adj_p = pdo_functions.control_FDR(z_500_pos_pdo_neut_nino_p_fast, 101, 237, 0.1)
z_500_neg_pdo_neut_nino_adj_p = pdo_functions.control_FDR(z_500_neg_pdo_neut_nino_p_fast, 101, 237, 0.1)
z_500_pos_oni_adj_p = pdo_functions.control_FDR(z_500_pos_oni_p_fast, 101, 237, 0.1)
z_500_neg_oni_adj_p = pdo_functions.control_FDR(z_500_neg_oni_p_fast, 101, 237, 0.1)
z_500_pos_nino_adj_p = pdo_functions.control_FDR(z_500_pos_nino_p_fast, 101, 237, 0.1)
z_500_neg_nino_adj_p = pdo_functions.control_FDR(z_500_neg_nino_p_fast, 101, 237, 0.1)

In [ ]:
wind_200_pos_pdo_sig = np.where(wind_200_pos_pdo_p_fast < wind_200_pos_pdo_adj_p, 1, 0)
wind_200_neg_pdo_sig = np.where(wind_200_neg_pdo_p_fast < wind_200_neg_pdo_adj_p, 1, 0)
wind_200_pos_pdo_neut_nino_sig = np.where(wind_200_pos_pdo_neut_nino_p_fast < wind_200_pos_pdo_neut_nino_adj_p, 1, 0)
wind_200_neg_pdo_neut_nino_sig = np.where(wind_200_neg_pdo_neut_nino_p_fast < wind_200_neg_pdo_neut_nino_adj_p, 1, 0)
wind_200_pos_oni_sig = np.where(wind_200_pos_oni_p_fast < wind_200_pos_oni_adj_p, 1, 0)
wind_200_neg_oni_sig = np.where(wind_200_neg_oni_p_fast < wind_200_neg_oni_adj_p, 1, 0)
wind_200_pos_nino_sig = np.where(wind_200_pos_nino_p_fast < wind_200_pos_nino_adj_p, 1, 0)
wind_200_neg_nina_sig = np.where(wind_200_neg_nino_p_fast < wind_200_neg_nino_adj_p, 1, 0)

wind_850_pos_pdo_sig = np.where(wind_850_pos_pdo_p_fast < wind_850_pos_pdo_adj_p, 1, 0)
wind_850_neg_pdo_sig = np.where(wind_850_neg_pdo_p_fast < wind_850_neg_pdo_adj_p, 1, 0)
wind_850_pos_pdo_neut_nino_sig = np.where(wind_850_pos_pdo_neut_nino_p_fast < wind_850_pos_pdo_neut_nino_adj_p, 1, 0)
wind_850_neg_pdo_neut_nino_sig = np.where(wind_850_neg_pdo_neut_nino_p_fast < wind_850_neg_pdo_neut_nino_adj_p, 1, 0)
wind_850_pos_oni_sig = np.where(wind_850_pos_oni_p_fast < wind_850_pos_oni_adj_p, 1, 0)
wind_850_neg_oni_sig = np.where(wind_850_neg_oni_p_fast < wind_850_neg_oni_adj_p, 1, 0)
wind_850_pos_nino_sig = np.where(wind_850_pos_nino_p_fast < wind_850_pos_nino_adj_p, 1, 0)
wind_850_neg_nina_sig = np.where(wind_850_neg_nino_p_fast < wind_850_neg_nino_adj_p, 1, 0)

z_500_pos_pdo_sig = np.where(z_500_pos_pdo_p_fast < z_500_pos_pdo_adj_p, 1, 0)
z_500_neg_pdo_sig = np.where(z_500_neg_pdo_p_fast < z_500_neg_pdo_adj_p, 1, 0)
z_500_pos_pdo_neut_nino_sig = np.where(z_500_pos_pdo_neut_nino_p_fast < z_500_pos_pdo_neut_nino_adj_p, 1, 0)
z_500_neg_pdo_neut_nino_sig = np.where(z_500_neg_pdo_neut_nino_p_fast < z_500_neg_pdo_neut_nino_adj_p, 1, 0)
z_500_pos_oni_sig = np.where(z_500_pos_oni_p_fast < z_500_pos_oni_adj_p, 1, 0)
z_500_neg_oni_sig = np.where(z_500_neg_oni_p_fast < z_500_neg_oni_adj_p, 1, 0)
z_500_pos_nino_sig = np.where(z_500_pos_nino_p_fast < z_500_pos_nino_adj_p, 1, 0)
z_500_neg_nina_sig = np.where(z_500_neg_nino_p_fast < z_500_neg_nino_adj_p, 1, 0)

In [ ]:
# Land mask
land_110 = regionmask.defined_regions.natural_earth_v4_1_0.land_110
land = xr.where(land_110.mask_3D(wind_850_pos_pdo, lon_name='longitude', lat_name='latitude')==True, 1, np.nan).squeeze()

In [ ]:
fig, ax = plt.subplots(4, 3, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ax = [ax[0, 0], ax[0, 1], ax[0, 2], \
      ax[1, 0], ax[1, 1], ax[1, 2], \
      ax[2, 0], ax[2, 1], ax[2, 2], \
      ax[3, 0], ax[3, 1], ax[3, 2]]

lon = z_500['longitude']
lat = z_500['latitude']

letters = ['a', 'b', 'c', \
           'd', 'e', 'f', \
           'g', 'h', 'i', \
           'k', 'l', 'm']

wind_200_bounds = [-3, -2.7, -2.4, -2.1, -1.8, -1.5, -1.2, -0.9, -0.6, -0.3, -0.06, 0.06, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3]
wind_200_norm = c.BoundaryNorm(wind_200_bounds, plt.get_cmap('cmr.fusion_r').N)

z_500_bounds = [-100, -90, -80, -70, -60, -50, -40, -30, -20, -10, -2, 2, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
z_500_norm = c.BoundaryNorm(z_500_bounds, plt.get_cmap('cmr.fusion_r').N)

wind_850_bounds = [-0.7, -0.63, -0.56, -0.49, -0.42, -0.35, -0.28, -0.21, -0.14, -0.07, -0.014, 0.014, 0.07, 0.14, 0.21, 0.28, 0.35, 0.42, 0.49, 0.56, 0.63, 0.7]
wind_850_norm = c.BoundaryNorm(wind_850_bounds, plt.get_cmap('cmr.fusion_r').N)

for i in range(12):
    ax[i].coastlines(lw=0.25, color='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0)
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].set_facecolor('xkcd:light gray')
    ax[i].text(-2700000, 1420000, letters[i], fontweight='normal', size=10)

ax[2].pcolormesh(lon, lat, wind_200_pos_pdo.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_200_norm)
ax[5].pcolormesh(lon, lat, wind_200_pos_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_200_norm)
ax[8].pcolormesh(lon, lat, wind_200_neg_pdo.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_200_norm)
ax[11].pcolormesh(lon, lat, wind_200_neg_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_200_norm)

ax[1].pcolormesh(lon, lat, z_500_pos_pdo.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=z_500_norm)
ax[4].pcolormesh(lon, lat, z_500_pos_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=z_500_norm)
ax[7].pcolormesh(lon, lat, z_500_neg_pdo.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=z_500_norm)
ax[10].pcolormesh(lon, lat, z_500_neg_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=z_500_norm)

ax[0].pcolormesh(lon, lat, wind_850_pos_pdo.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_850_norm)
ax[3].pcolormesh(lon, lat, wind_850_pos_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_850_norm)
ax[6].pcolormesh(lon, lat, wind_850_neg_pdo.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_850_norm)
ax[9].pcolormesh(lon, lat, wind_850_neg_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_850_norm)

q = ax[0].quiver(lon[::8], lat[::8], u_850_pos_pdo.mean(dim='time').values[::8, ::8], v_850_pos_pdo.mean(dim='time').values[::8, ::8], width=0.002, scale=10, transform=ccrs.PlateCarree())
ax[3].quiver(lon[::8], lat[::8], u_850_pos_pdo_neut_oni.mean(dim='time').values[::8, ::8], v_850_pos_pdo_neut_oni.mean(dim='time').values[::8, ::8], width=0.002, scale=10, transform=ccrs.PlateCarree())
ax[6].quiver(lon[::8], lat[::8], u_850_neg_pdo.mean(dim='time').values[::8, ::8], v_850_neg_pdo.mean(dim='time').values[::8, ::8], width=0.002, scale=10, transform=ccrs.PlateCarree())
ax[9].quiver(lon[::8], lat[::8], u_850_neg_pdo_neut_oni.mean(dim='time').values[::8, ::8], v_850_neg_pdo_neut_oni.mean(dim='time').values[::8, ::8], width=0.002, scale=10, transform=ccrs.PlateCarree())

# ax[0].contourf(lon, lat, wind_850_pos_pdo_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[3].contourf(lon, lat, wind_850_pos_pdo_neut_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[6].contourf(lon, lat, wind_850_neg_pdo_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[9].contourf(lon, lat, wind_850_neg_pdo_neut_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])

# ax[1].contourf(lon, lat, z_500_pos_pdo_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[4].contourf(lon, lat, z_500_pos_pdo_neut_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[7].contourf(lon, lat, z_500_neg_pdo_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[10].contourf(lon, lat, z_500_neg_pdo_neut_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])

# ax[2].contourf(lon, lat, wind_200_pos_pdo_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[5].contourf(lon, lat, wind_200_pos_pdo_neut_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[8].contourf(lon, lat, wind_200_neg_pdo_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[11].contourf(lon, lat, wind_200_neg_pdo_neut_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])

ax[0].quiverkey(q, X=0.94, Y=0.1, U=0.5, label='0.5 m/s', fontproperties={'weight':'normal', 'size':4}, labelpos='S',labelsep=0.05, zorder=100)

ax[0].set_title('850 mb wind', pad=3, fontsize=10)
ax[1].set_title('500 mb Z', pad=3, fontsize=10)
ax[2].set_title('200 mb wind', pad=3, fontsize=10)

ax[0].text(-3700000, -100000, f"(+) PDO\nn={len(z_500_pos_pdo)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[3].text(-4000000, -400000, f"(+) PDO &\nneutral ONI\nn={len(z_500_pos_pdo_neut_oni)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[6].text(-3700000, -100000, f"(-) PDO\nn={len(z_500_neg_pdo)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[9].text(-4000000, -400000, f"(-) PDO &\nneutral ONI\nn={len(z_500_neg_pdo_neut_oni)}", fontweight='normal', size=10, rotation='horizontal', ha='center')

cax = plt.axes([0.02, .23, 0.29, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=wind_850_norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', \
                   ticks=[-.7, -0.35, 0, 0.35, 0.7])
cbar.set_label(r'm s$^{-1}$', size=10, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.25)
cbar.outline.set_color('black')
# cbar.outline.set_visible(False)

cax2 = plt.axes([0.355, .23, 0.29, 0.02])
cbar2 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=z_500_norm), cax=cax2, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-100, -50, 0, 50, 100])
cbar2.set_label(r'm', size=10, fontweight='normal', color='black')
cbar2.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar2.outline.set_linewidth(0.25)
cbar2.outline.set_color('black')
# cbar.outline.set_visible(False)

cax3 = plt.axes([0.695, .23, 0.29, 0.02])
cbar3 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=wind_200_norm), cax=cax3, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-3, -1.5, 0, 1.5, 3])
cbar3.set_label(r'm s$^{-1}$', size=10, fontweight='normal', color='black')
cbar3.ax.tick_params(which='both', labelsize=10, width=0.25, length=0, labelcolor='black')
cbar3.outline.set_linewidth(0.25)
cbar3.outline.set_color('black')
# cbar.outline.set_visible(False)

plt.subplots_adjust(left=0,
                    bottom=0.25, 
                    right=1., 
                    top=1.3, 
                    wspace=0.05, 
                    hspace=0.05)

In [ ]:
fig, ax = plt.subplots(4, 3, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ax = [ax[0, 0], ax[0, 1], ax[0, 2], \
      ax[1, 0], ax[1, 1], ax[1, 2], \
      ax[2, 0], ax[2, 1], ax[2, 2], \
      ax[3, 0], ax[3, 1], ax[3, 2]]

lon = z_500['longitude']
lat = z_500['latitude']

letters = ['a', 'b', 'c', \
           'd', 'e', 'f', \
           'g', 'h', 'i', \
           'k', 'l', 'm']

wind_200_bounds = [-5, -4.5, -4, -3.5, -3, -2.5, -2, -1.5, -1, -0.5, -0.1, 0.1, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
wind_200_norm = c.BoundaryNorm(wind_200_bounds, plt.get_cmap('cmr.fusion_r').N)

z_500_bounds = [-150, -135, -120, -105, -90, -75, -60, -45, -30, -15, -3, 3, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150]
z_500_norm = c.BoundaryNorm(z_500_bounds, plt.get_cmap('cmr.fusion_r').N)

wind_850_bounds = [-1.2, -1.05, -0.9, -0.75, -0.6, -0.45, -0.3, -0.15, -0.06, 0.06, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05, 1.2]
wind_850_norm = c.BoundaryNorm(wind_850_bounds, plt.get_cmap('cmr.fusion_r').N)

for i in range(12):
    ax[i].coastlines(lw=0.25, color='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0)
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].set_facecolor('xkcd:light gray')
    ax[i].text(-2700000, 1420000, letters[i], fontweight='normal', size=10)

ax[2].pcolormesh(lon, lat, wind_200_pos_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_200_norm)
ax[5].pcolormesh(lon, lat, wind_200_pos_nino.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_200_norm)
ax[8].pcolormesh(lon, lat, wind_200_neg_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_200_norm)
ax[11].pcolormesh(lon, lat, wind_200_neg_nina.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_200_norm)

ax[1].pcolormesh(lon, lat, z_500_pos_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=z_500_norm)
ax[4].pcolormesh(lon, lat, z_500_pos_nino.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=z_500_norm)
ax[7].pcolormesh(lon, lat, z_500_neg_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=z_500_norm)
ax[10].pcolormesh(lon, lat, z_500_neg_nina.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=z_500_norm)

ax[0].pcolormesh(lon, lat, wind_850_pos_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_850_norm)
ax[3].pcolormesh(lon, lat, wind_850_pos_nino.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_850_norm)
ax[6].pcolormesh(lon, lat, wind_850_neg_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_850_norm)
ax[9].pcolormesh(lon, lat, wind_850_neg_nina.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=wind_850_norm)

q = ax[0].quiver(lon[::8], lat[::8], u_850_pos_oni.mean(dim='time').values[::8, ::8], v_850_pos_oni.mean(dim='time').values[::8, ::8], width=0.002, scale=10, transform=ccrs.PlateCarree())
ax[3].quiver(lon[::8], lat[::8], u_850_pos_nino.mean(dim='time').values[::8, ::8], v_850_pos_nino.mean(dim='time').values[::8, ::8], width=0.002, scale=10, transform=ccrs.PlateCarree())
ax[6].quiver(lon[::8], lat[::8], u_850_neg_oni.mean(dim='time').values[::8, ::8], v_850_neg_oni.mean(dim='time').values[::8, ::8], width=0.002, scale=10, transform=ccrs.PlateCarree())
ax[9].quiver(lon[::8], lat[::8], u_850_neg_nina.mean(dim='time').values[::8, ::8], v_850_neg_nina.mean(dim='time').values[::8, ::8], width=0.002, scale=10, transform=ccrs.PlateCarree())

ax[0].quiverkey(q, X=0.94, Y=0.1, U=0.5, label='0.5 m/s', fontproperties={'weight':'normal', 'size':4}, labelpos='S',labelsep=0.05, zorder=100)

# ax[0].contourf(lon, lat, wind_850_pos_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[3].contourf(lon, lat, wind_850_pos_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[6].contourf(lon, lat, wind_850_neg_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[9].contourf(lon, lat, wind_850_neg_nina_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])

# ax[1].contourf(lon, lat, z_500_pos_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[4].contourf(lon, lat, z_500_pos_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[7].contourf(lon, lat, z_500_neg_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[10].contourf(lon, lat, z_500_neg_nina_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])

# ax[2].contourf(lon, lat, wind_200_pos_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[5].contourf(lon, lat, wind_200_pos_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[8].contourf(lon, lat, wind_200_neg_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[11].contourf(lon, lat, wind_200_neg_nina_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])

ax[0].set_title('850 mb wind', pad=3, fontsize=10)
ax[1].set_title('500 mb Z', pad=3, fontsize=10)
ax[2].set_title('200 mb wind', pad=3, fontsize=10)

ax[0].text(-3700000, -100000, f"(+) ONI\nn={len(z_500_pos_oni)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[3].text(-4000000, -400000, f"El Ni\u00f1o\nn={len(z_500_pos_nino)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[6].text(-3700000, -100000, f"(-) ONI\nn={len(z_500_neg_oni)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[9].text(-4000000, -400000, f"La Ni\u00f1a\nn={len(z_500_neg_nina)}", fontweight='normal', size=10, rotation='horizontal', ha='center')

cax = plt.axes([0.02, .23, 0.29, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=wind_850_norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', \
                   ticks=[-1.2, -0.6, 0, 0.6, 1.2])
cbar.set_label(r'm s$^{-1}$', size=10, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.25)
cbar.outline.set_color('black')
# cbar.outline.set_visible(False)

cax2 = plt.axes([0.355, .23, 0.29, 0.02])
cbar2 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=z_500_norm), cax=cax2, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-150, -75, 0, 75, 150])
cbar2.set_label(r'm', size=10, fontweight='normal', color='black')
cbar2.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar2.outline.set_linewidth(0.25)
cbar2.outline.set_color('black')
# cbar.outline.set_visible(False)

cax3 = plt.axes([0.695, .23, 0.29, 0.02])
cbar3 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=wind_200_norm), cax=cax3, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-5, -2.5, 0, 2.5, 5])
cbar3.set_label(r'm s$^{-1}$', size=10, fontweight='normal', color='black')
cbar3.ax.tick_params(which='both', labelsize=10, width=0.25, length=0, labelcolor='black')
cbar3.outline.set_linewidth(0.25)
cbar3.outline.set_color('black')
# cbar.outline.set_visible(False)

plt.subplots_adjust(left=0,
                    bottom=0.25, 
                    right=1., 
                    top=1.3, 
                    wspace=0.05, 
                    hspace=0.05)